In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
#import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm, trange

import dtt_fiber_processing

pd.options.mode.chained_assignment = None

In [133]:
data_dir = Path('Z:\\3_Histology\\Qupath Projects\\export measurements Test65\\')
query_file, data_files = dtt_fiber_processing.parse_datafile_paths(data_dir)
#data_file = data_files[0] # We're going to work with one file for now

data_file = data_dir.joinpath('24_3_ACP.csv')
data = pd.read_csv(data_file)


In [134]:
tree, id_map, structure_map = dtt_fiber_processing.get_atlas_components()

In [136]:
left_data, right_data = dtt_fiber_processing.split_data(data)

#left_data = left_data[left_data.Classification != 'root']
#right_data = right_data[right_data.Classification != 'root']

left_data.reset_index(inplace=True, drop=True)
right_data.reset_index(inplace=True, drop=True)

fiber_key = [ids for ids in id_map if 'fiber' in ids][0]
id_map['fiber_tracts'] = id_map[fiber_key]

In [137]:
left_acronyms = left_data['Classification']
right_acronyms = right_data['Classification']
left_ids = [id_map[region] for region in left_acronyms]
right_ids = [id_map[region] for region in right_acronyms]

In [138]:
left_data.insert(1, 'ID', left_ids)
right_data.insert(1, 'ID', right_ids)

In [139]:
left_data['Artifact Size'] = left_data['Artifact Size'].fillna(0)
right_data['Artifact Size'] = right_data['Artifact Size'].fillna(0)

In [18]:
for row in trange(left_data.shape[0]):
    region_row = left_data.iloc[row]
    region_id = region_row['ID']
    
    region_path = structure_map[region_id]
    
    region_artifact = region_row['Artifact Size']
    for parent_region in region_path:
        data_mask = left_data['ID'] == parent_region
        new_artifact = left_data.loc[data_mask]['Artifact Size'] - region_artifact
        left_data.loc[data_mask]['Artifact Size'] = new_artifact
        

100%|██████████| 64/64 [00:00<00:00, 376.47it/s]


In [150]:
region_path = structure_map[154][1::]

data_mask = left_data['ID'] == region_path[0]

In [151]:
data_mask

0     False
1     False
2     False
3     False
4     False
      ...  
59    False
60    False
61    False
62    False
63    False
Name: ID, Length: 64, dtype: bool